In [2]:
#Import libraries
from IPython.display import clear_output
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd

In [1]:
#Set path of the directory
path = ('eurlex_html_EN_NOT/')
base = Path(path)

#Set total count of files for progress indicator
total_file_len = len([x for x in base.iterdir()])

docs = []
for count, filename in enumerate(base.iterdir()):
    #Initialize beautiful soup
    with open(filename, 'r') as file:
        contents = file.read()
        soup = BeautifulSoup(contents, 'lxml')

        #Fetch Main Text
        tags = soup.findAll("div", {"class": "texte"})
        main_text = []
        for para in tags:
            main_text.extend(((para.text).lower().replace(',','').replace('.', '')).split())

        #Fetch Classes
        tags = soup.findAll('li')
        classes = []
        for st in tags:
            for i in st.findAll('strong'):
                if i.text == 'Subject matter:':
                    classes = (st.text.strip()).split('\n')
                    classes = [x.lower().replace(' ', '_') for x in classes if (x != ', ') and (x != '') and (x != ',')][1:]
        
        #Append text and classes together in multi-D list
        docs.append([main_text, classes])
    
    #Display progress %
    clear_output(wait = True)
    print("Progress: {:.2f}".format(count/total_file_len * 100), "%")

In [22]:
#Save the output in a dataframe
df = pd.DataFrame(docs)
for row in range(len(df)):
    df.at[row, 0] = ' '.join(df.at[row, 0])
    df.at[row, 1] = ', '.join(df.at[row, 1])

In [12]:
#Write to .csv
df.to_csv('output.csv', header=False, index=False)